In [ ]:
import torch
import os
import cv2
import shutil
import argparse
import numpy as np

from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision.utils import save_image
from torchvision import transforms as T
from torchvision.transforms.functional import to_pil_image
from threading import Thread
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm

Set environment

In [ ]:
#{"username":"lindsaymorel","key":"9a094067d35d2f2b37e2362eda70c1ce"}

os.environ["KAGGLE_USERNAME"] = "lindsaymorel"
os.environ["KAGGLE_KEY"] = "9a094067d35d2f2b37e2362eda70c1ce"
if not os.path.exists("aisegmentcom-matting-human-datasets.zip"):
  !kaggle datasets download -d laurentmih/aisegmentcom-matting-human-datasets/clip_img
  !unzip aisegmentcom-matting-human-datasets

In [ ]:
 !unzip aisegmentcom-matting-human-datasets

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)


Mounted at /content/drive


Parse Arguments

In [ ]:
#add description
parser =argparse.ArgumentParser(description='N/a')
matting_type = parser.add_mutually_exclusive_group(required=True)
matting_type.add_argument('--video', action='store_true')
matting_type.add_argument('--image', action='store_true')
parser.add_argument('--src', type=str, required=True)
parser.add_argument('--bgr', type=str, required=True)
parser.add_argument('--dst', type=str, required=True)


_StoreAction(option_strings=['--dst'], dest='dst', nargs=None, const=None, default=None, type=<class 'str'>, choices=None, help=None, metavar=None)

Create Model 
And Load Datasets

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
# model version may need to be changed
model = torch.hub.load('pytorch/vision:v0.6.0', model='fcn_resnet101', pretrained=True)
model.to(device)
model.eval()


Get the frames from video, and convert array of frames to video

In [ ]:
def get_frames(path):
  cap= cv2.VideoCapture(path)
  fps=cap.get(5)
  i=0
  frames=[]
  while(cap.isOpened()):
      ret, frame = cap.read()
      if ret == False:
          break
          #this rotates the frame
      #frame=frame.transpose(1,0,2)
      cv2.imwrite('frame_' + str(i) + '.jpg',frame) #stored frames as jpgs
      frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) 
      frames.append(frame)
      i+=1
  cap.release()
  cv2.destroyAllWindows()
  output=np.stack(frames,axis=0) #size=(N,H,W,3)
  #output=torch.tensor(output)
  return fps,output

def frames2video(fps,output, name):
  output=output.numpy()
  out=cv2.VideoWriter(name, cv2.VideoWriter_fourcc(*'mp4v'), fps, (output.shape[2],output.shape[1]))
  for i in np.arange(output.shape[0]):
    img=cv2.cvtColor(output[i,:,:,:], cv2.COLOR_RGB2BGR)
    out.write(img)
  out.release()

Save session


In [ ]:
import dill
from google.colab import drive

backup_dir = 'drive/My Drive/colab_sessions'
backup_file = 'notebook_env.db'
backup_path = backup_dir + '/' + backup_file

def init_drive():
  # create directory if not exist
  drive.mount('drive')
  if not os.path.exists(backup_dir):
    !mkdir backup_dir

def restart_kernel():
  os._exit(00)

def save_session():
  init_drive()
  dill.dump_session(backup_path)

def load_session():
  init_drive()
  dill.load_session(backup_path)

Train Model


Evaluate model and composite video 

In [ ]:
def predict(model,image,device):
  if torch.cuda.is_available():
    device = "cuda" 
  else:
    device = "cpu"
  mean = [0.485, 0.456, 0.406]
  std = [0.229, 0.224, 0.225]
  process=T.Compose([T.ToTensor(), T.Normalize(mean,std)])
  img=process(image)
  batch=img.unsqueeze(0)
  batch.to(device)
  with torch.no_grad():
    output=model(batch)['out'][0]
    # output=nn.functional.softmax(output, dim=0)
    output=torch.argmax(output,dim=0).cpu().numpy()
  return output

#REQUIRES: Foreground, Background and Segmentation all have the same shape
def composite(foreground, background, segmentation):
  print("sheesh", foreground.shape, background.shape, segmentation.shape)
  assert foreground.shape == background.shape
  assert foreground.shape == segmentation.shape
  fgr = foreground * segmentation
  seg = ~segmentation
  
  bgr = background * seg
  print(type(fgr), fgr.shape, segmentation.shape, bgr.shape, type(segmentation), type(bgr))
  #s = np.asarray(segmentation)
  #plt.imshow(cv2.cvtColor(segmentation, cv2.COLOR_BGR2RGB))
  #plt.imshow(cv2.cvtColor(seg, cv2.COLOR_BGR2RGB))
  ##plt.imshow(cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB))
  #plt.imshow(cv2.cvtColor(fgr, cv2.COLOR_BGR2RGB))
  f = fgr > 0
  b = bgr > 0
  print(f)
  cv2.imwrite('/content/drive/MyDrive/EECS 442 Final Project/out/mask1.jpg', f.astype(int) * 255)
  cv2.imwrite('/content/drive/MyDrive/EECS 442 Final Project/out/inverted_mask1.jpg', b.astype(int) * 255)

  cv2.imwrite('/content/drive/MyDrive/EECS 442 Final Project/out/foreground1.jpg', fgr )
  
  cv2.imwrite('/content/drive/MyDrive/EECS 442 Final Project/out/background1.jpg', bgr)
  return fgr + bgr

#Image is tensor of the composited image, name is the filename, and to_drve
# is a bool if the file is to be saved to google drive
def download_image(image, name, to_drive):
  to_pil_image(image.cpu()).save(name)
  if to_drive:
    files.download(name)

#Frames is the composited frame of the video as a tensor
def download_video(frames, fps, name, to_drive):
  frames2video(fps, frames, name)
  if to_drive:
    files.download(name)


In [ ]:
# src is video source, bgr is background image source, dst is destination to save composite
def composite_video(model, srcPath, bgrPath, dst):
  bgr = cv2.imread(bgrPath)
  fps, frames = get_frames(srcPath)
  N, H, W, _ = frames.shape
  print("this is the shape of frames", frames.shape)
  out = cv2.VideoWriter(dst, cv2.VideoWriter_fourcc(*'mp4v'), fps, (W, H))
  print(len(frames), fps)
  #todo see if any transforms are needed
  #background = Image.open(bgr).convert('RGB')
  #resize background image to be the same size as the video 
  background = cv2.resize(bgr, (W, H), interpolation=cv2.INTER_AREA)
  background = cv2.cvtColor(background, cv2.COLOR_BGR2RGB)
  i = 0
  for frame in tqdm(frames):  
    #pred = predict(model, frames[i], device)
    pred = predict(model, frame, device)
    mask = pred == 15
    mask = np.repeat(mask[:, :, np.newaxis], 3, axis = 2)
    
    #comp = composite(frames[i], background, mask)
    comp = composite(frame, background, mask)
    # comp=background.copy()
    # comp[mask]=frame[mask]
    print('comp', comp.shape)
    #todo check that comp comes out as a tensor
    img=cv2.cvtColor(comp, cv2.COLOR_RGB2BGR)#[i,:,:], cv2.COLOR_RGB2BGR)
    print("comp shape", img.shape)
    out.write(img)
    i += 1
    #if i > len(frames)/10:
    # break
    if i < 2:
      print('frame', type(frames[i]), frames[i].shape)
      print('comp', type(comp), comp.shape)
      print('background', type(background), background.shape)
  out.release()

In [ ]:
def composite_image(model,srcPath, bgrPath, dst):
  src = cv2.imread(srcPath)
  bgr = cv2.imread(bgrPath)
  # bgr=cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
  print(type(src))
  prediction = predict(model, src, device)
  H, W, _ = src.shape
  mask = prediction == 15
  mask = np.repeat(mask[:, :, np.newaxis], 3, axis = 2)
  # background = Image.open(bgr).convert('RGB')
  background = cv2.resize(bgr, (W, H), interpolation=cv2.INTER_AREA)
  # download_image(composited, "output.jpg", True)
  composited = composite(src, background, mask)
  plt.imshow(cv2.cvtColor(composited, cv2.COLOR_BGR2RGB))
  out = cv2.imwrite(dst,composited)

In [ ]:

    
srcPath = '/content/drive/MyDrive/EECS 442 Final Project/img_set2/example.jpg'
#src = cv2.imread(path)
dst = '/content/drive/MyDrive/EECS 442 Final Project/out/composite_img3.jpg'
bgrPath = '/content/drive/MyDrive/EECS 442 Final Project/bgr/bgr1.jpg'
#bgr = cv2.imread(bgrPath)
 
#src = parser.src
#bgr = parser.bgr
#dst = parser.dst

#Output to video format
#if parser.video:
#  composite_video(src, bgr, dst)
#output to image format
#elif parser.image:
composite_image(model,srcPath, bgrPath, dst)


In [ ]:
bgrPath = '/content/drive/MyDrive/EECS 442 Final Project/bgr/bgr3.jpg'
srcPath = '/content/drive/MyDrive/EECS 442 Final Project/vid/vs3_360.mp4'
dst = '/content/drive/MyDrive/EECS 442 Final Project/out/composite_vid3.mp4'
composite_video(model, srcPath, bgrPath, dst)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import moviepy.editor as mp
clip = mp.VideoFileClip("/content/drive/MyDrive/EECS 442 Final Project/vid/vs3.mp4")
clip_resized = clip.resize(height=360) # make the height 360px ( According to moviePy documenation The width is then computed so that the width/height ratio is conserved.)
clip_resized.write_videofile("/content/drive/MyDrive/EECS 442 Final Project/vid/vs3_360.mp4")